In [22]:
def BWT(T):#Burrows Wheeler transform of sequence T
    T=T+'$'
    s=[]
    s.append(T)
    for i in range(len(T)-1):
        T=T[1:]+T[0]
        s.append(T)
    s=sorted(s)
    transformed=''
    for t in s:
        transformed+=t[-1]
    return transformed
BWT("CATATG")

'GCT$TAA'

In [59]:
def invBWT(BWT): #inverse BW transformation  of sequence BWT. return the first occurrence of each character 
    d={'$':0,'A':0,'C':0,'G':0,'T':0}
    for s in BWT:
        d[s]+=1
    FstOcc=[0,1,1+d['A'],1+d['A']+d['C'],1+d['A']+d['C']+d['G']] #$,'A','C','G','T'
    return FstOcc,d
invBWT('GCT$TAAA')

([0, 1, 4, 5, 6], {'$': 1, 'A': 3, 'C': 1, 'G': 1, 'T': 2})

In [22]:
#build index of BWT:
BWT='GCT$TAA'
d={'$':0,'A':0,'C':0,'G':0,'T':0} # for calculating the nucleotide occurrence in BWT
BWTplus=[] #BWT with occurance index
for a in BWT:
    d[a]+=1
    BWTplus.append((a,d[a]))
dic=invBWT(BWT)
 
def LF(i,BWTplus,dic): #last to front function 
    if BWTplus[i-1][0]=='A': 
        return 1+ BWTplus[i-1][1]
    if BWTplus[i-1][0]=='C': 
        return 1+ dic['A']+BWTplus[i-1][1]
    if BWTplus[i-1][0]=='G': 
        return 1+ dic['A']+dic['C']+BWTplus[i-1][1]
    if BWTplus[i-1][0]=='T': 
        return 1+ dic['A']+dic['C']+dic['G']+BWTplus[i-1][1]


6

In [58]:
# build the count table of BWT:
    
def counttable(BWT):

    import numpy as np
    L=len(BWT)
    table=np.zeros((L+1,5),dtype=int)
    d={'$':0,'A':1,'C':2,'G':3,'T':4}
    count=np.array((0,0,0,0,0))

    for i in range(L):
        count[d[BWT[i]]]+=1
        table[i+1]=count

    return table
counttable('CTTGTA')

array([[0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 0, 2],
       [0, 0, 1, 1, 2],
       [0, 0, 1, 1, 3],
       [0, 1, 1, 1, 3]])

In [39]:
#Build original index of T for BWT, which was sorted with the partial suffix array
#Given T
T='panamabananas'
def buildindex(T):
    suffixlist=[]
    T=T+'$'
    for i in range(len(T)):
        suffixlist.append((T[i:],i))
    sortedsuffix=sorted(suffixlist)
    import pandas as pd
    ss=pd.DataFrame(sortedsuffix)
    index=ss.loc[:,1]
    return index
buildindex(T)

0     13
1      5
2      3
3      1
4      7
5      9
6     11
7      6
8      4
9      2
10     8
11    10
12     0
13    12
Name: 1, dtype: int64

In [62]:
# Given read, T,

T="ACAACGTT"
bwt=BWT(T)
FirstOcc,numOfNu=invBWT(bwt)
count=counttable(bwt)
read='T'
d={'$':0,'A':1,'C':2,'G':3,'T':4} # symbol (nuleutide) mapping to index
IND=buildindex(T)
#initialize the top and bottom pointer
index=d[read[-1]]
top=FirstOcc[index]
if index==4:
    bottom=FirstOcc[index]+numOfNu['T']-1
else:
    bottom=FirstOcc[index+1]-1

for i in range(1,len(read)):
    index=d[read[-1-i]]
    top=FirstOcc[index]+count[top,index]
    bottom=FirstOcc[index]+count[bottom+1,index]-1

if bottom >=top: # find matches
    for i in range(top, bottom+1):
        print(IND.loc[i])




7
6


In [71]:
#use hash to do the mapping:
#build hash table for T, Given length of read, allowing 3 mismatches
T='AAGAGCAGGGGATCTCCCCCCTCCACAGTCTTAGTACCTCCTAATAATGGAGTGGGCTCTGATCAATGTGGGTTATAGACAGTGCGCACGCATATCAAAT'
read='AAGAGCAGGGGATCTCCCCCCTCCACAGTCTTAGTACCTCCTAATAATGG'

def buildhash(T):
    k=12#len(read)//4
    d={}
    for i in range(len(T)-k+1):
        kmer=T[i:i+k]
        if kmer not in d:
            d[kmer]=[i]
        else:
            d[kmer].append(i)
    return d
hashtable=buildhash(T)

In [88]:
#map reads using the hash table
T='AGTGGGCTCTGATCAATGTGGGTTATAGACAGTGCGCACGCATATCAAATAAGAGCAGGGGATCTCCCCCCTCCACAGTCTTAGTACCTCCTAATAATGG'
read='AAGAGCAGGGGATCTCCCCCCTCCACAGTCTTAGTACCTCCTAATAATGG'
k=12
hashtable=buildhash(T)
read_alignment_locations=[]
output_read_pair=[]

def hash_read_mapping(read,T,read_alignment_locations,output_read_pair):
    def dH(s,t): # Hamming distance (if length of s and t are not equal, return -1)
        if len(s)!=len(t):
            return -1
        d=0 #initialize distance =0
        for i in range(0,len(s)):
            if s[i] != t[i]:
                d+=1
        return d
    r1=read[0:12]
    r2=read[13:25]
    r3=read[26:38]
    r4=read[38:]
    L=len(T)
    if r1 in hashtable:
        for i in hashtable[r1]:#compare read[12:]
            d=dH(T[i+12:min(i+50,L)],read[12:])
            if d>=0 and d<=3:
                read_alignment_locations.append(i)
                output_read_pair.append(read)
    elif r2 in hashtable:
        #compare read[0:13] and read[25:]
        for i in hashtable[r2]:
            d=dH(T[max(0,i-13):i+37],read)
            if d>=0 and d<=3:
                read_alignment_locations.append(i-13)
                output_read_pair.append(read)
    elif r3 in hashtable:
        #compare read[0:26] and read[38:]
        for i in hashtable[r3]:
            d=dH(T[max(0,i-26):i+24],read)
            if d>=0 and d<=3:
                read_alignment_locations.append(i-26)
                output_read_pair.append(read)
    elif r4 in hashtable:
        for i in hashtable[r4]:#compare read[0:38] 
            d=dH(T[max(0,i-38):i],read[0:38])
            if d>=0 and d<=3:
                read_alignment_locations.append(i-38)
                output_read_pair.append(read)

hash_read_mapping(read,T,read_alignment_locations,output_read_pair)
read_alignment_locations,output_read_pair

([50], ['AAGAGCAGGGGATCTCCCCCCTCCACAGTCTTAGTACCTCCTAATAATGG'])

In [91]:
l=[2,1,4]
print(l.count)

<built-in method count of list object at 0x10b323048>
